In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

In [2]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
import dlib
import re

In [3]:
import cv2
from PIL import Image

# Define a few variables

In [4]:
basedir = './dataset_AMLS_22-23/celeba'
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels.csv'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Define a set of functions to be used to extract the features 

In [5]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [6]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

In [7]:
def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image

In [8]:
# This function was added by myself
def generate_dict_from_df(df, keys_col, values_col):
    zip_ = zip(df[keys_col], df[values_col])
    l = list(zip_)
    d = dict(l)
    return d

In [9]:
def extract_features_labels():
    """
    This funtion extracts the landmarks features for all images in the folder 'dataset/celeba'.
    It also extract the gender label for each image.
    :return:
        landmark_features:  an array containing 68 landmark points for each image in which a face was detected
        gender_labels:      an array containing the gender label (male=0 and female=1) for each image in
                            which a face was detected
    """
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
    target_size = None
    #labels_file = open(os.path.join(basedir, labels_filename), 'r')
    #lines = labels_file.readlines()
    labels_df = pd.read_csv(os.path.join(basedir, labels_filename), sep='\t')
    gender_labels = generate_dict_from_df(labels_df, 'img_name', 'gender')
    #smiling_labels = generate_dict_from_df(labels_df, 'file_name', 'smiling')
    #gender_labels = {line.split(',')[0] : int(line.split(',')[6]) for line in lines[2:]}
    if os.path.isdir(images_dir):
        all_features = []
        all_gender_labels = []
        #all_smiling_labels = []
        image_files = []
        i = 1
        for img_path in image_paths:
            print(f'Iteration {i} out of {len(image_paths)}')
            #file_name= img_path.split('.')[1].split('/')[-1]
            #file_name = str(img_path).replace('\\\\', 'mmmm')
            new_file_name = re.search('img\\\\(.*)', img_path).group(1)
            #new_img_path = re.search('\\(.*).png', img_path)
            #print(file_name)
            #print(new_file_name)

            # load image
            img = tf.keras.utils.img_to_array(
                tf.keras.utils.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
            features, _ = run_dlib_shape(img)
            if features is not None:
                all_features.append(features)
                all_gender_labels.append(gender_labels[new_file_name])
                #all_smiling_labels.append(smiling_labels[new_file_name])
                image_files.append(new_file_name)
            i += 1

    landmark_features = np.array(all_features)
    gender_labels = (np.array(all_gender_labels) + 1)/2 # simply converts the -1 into 0, so male=0 and female=1
    #smiling_labels = (np.array(all_smiling_labels) + 1)/2
    return landmark_features, gender_labels, image_files

# Create a dataset with all the 28x28 images

I will be creating a simple Convolutional Neural Network for this task. However, due to the relatively large size of the images, the CNN would have too many trainable parameters if the original size was used. Therefore, I will resize all the images (28x28 grayscale).

In [11]:
# Obtain a dataframe containing the images and labels
df = pd.read_csv('./dataset_AMLS_22-23/celeba/labels.csv', sep='\t')
df.drop(df.columns[0], axis=1, inplace=True)
df

,img_name,gender,smiling
0,0.jpg,-1,1
1,1.jpg,-1,1
2,2.jpg,1,-1
3,3.jpg,-1,-1
4,4.jpg,-1,-1
...,...,...,...
4995,4995.jpg,1,1
4996,4996.jpg,1,1
4997,4997.jpg,1,1
4998,4998.jpg,1,1


In [60]:
# Initialize images array (which will store the pixel values for all images)
images = np.ndarray((5000, 28, 28))

In [61]:
# Initialize the gender_labels list
gender_labels = []

In [62]:
# Go through each image in the celeba dataset
for i in range(5000):
    print(f'Iteration {i} out of 5000')
    image = Image.open(f'./dataset_AMLS_22-23/celeba/img/{i}.jpg')
    
    # Resize the image and convert it to grayscale
    resized_image = image.resize((28, 28))
    resized_image_array = np.array(resized_image)
    image_grayscale = cv2.cvtColor(resized_image_array, cv2.COLOR_RGB2GRAY)
    image_grayscale = image_grayscale/255  # Divide by the maximum pixel value, to provide scaling
    
    # Store the image array and the label associated with that image
    images[i] = image_grayscale
    gender_labels.append(int((int(df[df['img_name'] == f'{i}.jpg']['gender']) + 1)/2))

Iteration 0 out of 5000
Iteration 1 out of 5000
Iteration 2 out of 5000
Iteration 3 out of 5000
Iteration 4 out of 5000
Iteration 5 out of 5000
Iteration 6 out of 5000
Iteration 7 out of 5000
Iteration 8 out of 5000
Iteration 9 out of 5000
Iteration 10 out of 5000
Iteration 11 out of 5000
Iteration 12 out of 5000
Iteration 13 out of 5000
Iteration 14 out of 5000
Iteration 15 out of 5000
Iteration 16 out of 5000
Iteration 17 out of 5000
Iteration 18 out of 5000
Iteration 19 out of 5000
Iteration 20 out of 5000
Iteration 21 out of 5000
Iteration 22 out of 5000
Iteration 23 out of 5000
Iteration 24 out of 5000
Iteration 25 out of 5000
Iteration 26 out of 5000
Iteration 27 out of 5000
Iteration 28 out of 5000
Iteration 29 out of 5000
Iteration 30 out of 5000
Iteration 31 out of 5000
Iteration 32 out of 5000
Iteration 33 out of 5000
Iteration 34 out of 5000
Iteration 35 out of 5000
Iteration 36 out of 5000
Iteration 37 out of 5000
Iteration 38 out of 5000
Iteration 39 out of 5000
Iteration 

Iteration 369 out of 5000
Iteration 370 out of 5000
Iteration 371 out of 5000
Iteration 372 out of 5000
Iteration 373 out of 5000
Iteration 374 out of 5000
Iteration 375 out of 5000
Iteration 376 out of 5000
Iteration 377 out of 5000
Iteration 378 out of 5000
Iteration 379 out of 5000
Iteration 380 out of 5000
Iteration 381 out of 5000
Iteration 382 out of 5000
Iteration 383 out of 5000
Iteration 384 out of 5000
Iteration 385 out of 5000
Iteration 386 out of 5000
Iteration 387 out of 5000
Iteration 388 out of 5000
Iteration 389 out of 5000
Iteration 390 out of 5000
Iteration 391 out of 5000
Iteration 392 out of 5000
Iteration 393 out of 5000
Iteration 394 out of 5000
Iteration 395 out of 5000
Iteration 396 out of 5000
Iteration 397 out of 5000
Iteration 398 out of 5000
Iteration 399 out of 5000
Iteration 400 out of 5000
Iteration 401 out of 5000
Iteration 402 out of 5000
Iteration 403 out of 5000
Iteration 404 out of 5000
Iteration 405 out of 5000
Iteration 406 out of 5000
Iteration 40

Iteration 737 out of 5000
Iteration 738 out of 5000
Iteration 739 out of 5000
Iteration 740 out of 5000
Iteration 741 out of 5000
Iteration 742 out of 5000
Iteration 743 out of 5000
Iteration 744 out of 5000
Iteration 745 out of 5000
Iteration 746 out of 5000
Iteration 747 out of 5000
Iteration 748 out of 5000
Iteration 749 out of 5000
Iteration 750 out of 5000
Iteration 751 out of 5000
Iteration 752 out of 5000
Iteration 753 out of 5000
Iteration 754 out of 5000
Iteration 755 out of 5000
Iteration 756 out of 5000
Iteration 757 out of 5000
Iteration 758 out of 5000
Iteration 759 out of 5000
Iteration 760 out of 5000
Iteration 761 out of 5000
Iteration 762 out of 5000
Iteration 763 out of 5000
Iteration 764 out of 5000
Iteration 765 out of 5000
Iteration 766 out of 5000
Iteration 767 out of 5000
Iteration 768 out of 5000
Iteration 769 out of 5000
Iteration 770 out of 5000
Iteration 771 out of 5000
Iteration 772 out of 5000
Iteration 773 out of 5000
Iteration 774 out of 5000
Iteration 77

Iteration 1085 out of 5000
Iteration 1086 out of 5000
Iteration 1087 out of 5000
Iteration 1088 out of 5000
Iteration 1089 out of 5000
Iteration 1090 out of 5000
Iteration 1091 out of 5000
Iteration 1092 out of 5000
Iteration 1093 out of 5000
Iteration 1094 out of 5000
Iteration 1095 out of 5000
Iteration 1096 out of 5000
Iteration 1097 out of 5000
Iteration 1098 out of 5000
Iteration 1099 out of 5000
Iteration 1100 out of 5000
Iteration 1101 out of 5000
Iteration 1102 out of 5000
Iteration 1103 out of 5000
Iteration 1104 out of 5000
Iteration 1105 out of 5000
Iteration 1106 out of 5000
Iteration 1107 out of 5000
Iteration 1108 out of 5000
Iteration 1109 out of 5000
Iteration 1110 out of 5000
Iteration 1111 out of 5000
Iteration 1112 out of 5000
Iteration 1113 out of 5000
Iteration 1114 out of 5000
Iteration 1115 out of 5000
Iteration 1116 out of 5000
Iteration 1117 out of 5000
Iteration 1118 out of 5000
Iteration 1119 out of 5000
Iteration 1120 out of 5000
Iteration 1121 out of 5000
I

Iteration 1413 out of 5000
Iteration 1414 out of 5000
Iteration 1415 out of 5000
Iteration 1416 out of 5000
Iteration 1417 out of 5000
Iteration 1418 out of 5000
Iteration 1419 out of 5000
Iteration 1420 out of 5000
Iteration 1421 out of 5000
Iteration 1422 out of 5000
Iteration 1423 out of 5000
Iteration 1424 out of 5000
Iteration 1425 out of 5000
Iteration 1426 out of 5000
Iteration 1427 out of 5000
Iteration 1428 out of 5000
Iteration 1429 out of 5000
Iteration 1430 out of 5000
Iteration 1431 out of 5000
Iteration 1432 out of 5000
Iteration 1433 out of 5000
Iteration 1434 out of 5000
Iteration 1435 out of 5000
Iteration 1436 out of 5000
Iteration 1437 out of 5000
Iteration 1438 out of 5000
Iteration 1439 out of 5000
Iteration 1440 out of 5000
Iteration 1441 out of 5000
Iteration 1442 out of 5000
Iteration 1443 out of 5000
Iteration 1444 out of 5000
Iteration 1445 out of 5000
Iteration 1446 out of 5000
Iteration 1447 out of 5000
Iteration 1448 out of 5000
Iteration 1449 out of 5000
I

...

Iteration 3420 out of 5000
Iteration 3421 out of 5000
Iteration 3422 out of 5000
Iteration 3423 out of 5000
Iteration 3424 out of 5000
Iteration 3425 out of 5000
Iteration 3426 out of 5000
Iteration 3427 out of 5000
Iteration 3428 out of 5000
Iteration 3429 out of 5000
Iteration 3430 out of 5000
Iteration 3431 out of 5000
Iteration 3432 out of 5000
Iteration 3433 out of 5000
Iteration 3434 out of 5000
Iteration 3435 out of 5000
Iteration 3436 out of 5000
Iteration 3437 out of 5000
Iteration 3438 out of 5000
Iteration 3439 out of 5000
Iteration 3440 out of 5000
Iteration 3441 out of 5000
Iteration 3442 out of 5000
Iteration 3443 out of 5000
Iteration 3444 out of 5000
Iteration 3445 out of 5000
Iteration 3446 out of 5000
Iteration 3447 out of 5000
Iteration 3448 out of 5000
Iteration 3449 out of 5000
Iteration 3450 out of 5000
Iteration 3451 out of 5000
Iteration 3452 out of 5000
Iteration 3453 out of 5000
Iteration 3454 out of 5000
Iteration 3455 out of 5000
Iteration 3456 out of 5000
I

Iteration 3757 out of 5000
Iteration 3758 out of 5000
Iteration 3759 out of 5000
Iteration 3760 out of 5000
Iteration 3761 out of 5000
Iteration 3762 out of 5000
Iteration 3763 out of 5000
Iteration 3764 out of 5000
Iteration 3765 out of 5000
Iteration 3766 out of 5000
Iteration 3767 out of 5000
Iteration 3768 out of 5000
Iteration 3769 out of 5000
Iteration 3770 out of 5000
Iteration 3771 out of 5000
Iteration 3772 out of 5000
Iteration 3773 out of 5000
Iteration 3774 out of 5000
Iteration 3775 out of 5000
Iteration 3776 out of 5000
Iteration 3777 out of 5000
Iteration 3778 out of 5000
Iteration 3779 out of 5000
Iteration 3780 out of 5000
Iteration 3781 out of 5000
Iteration 3782 out of 5000
Iteration 3783 out of 5000
Iteration 3784 out of 5000
Iteration 3785 out of 5000
Iteration 3786 out of 5000
Iteration 3787 out of 5000
Iteration 3788 out of 5000
Iteration 3789 out of 5000
Iteration 3790 out of 5000
Iteration 3791 out of 5000
Iteration 3792 out of 5000
Iteration 3793 out of 5000
I

Iteration 4107 out of 5000
Iteration 4108 out of 5000
Iteration 4109 out of 5000
Iteration 4110 out of 5000
Iteration 4111 out of 5000
Iteration 4112 out of 5000
Iteration 4113 out of 5000
Iteration 4114 out of 5000
Iteration 4115 out of 5000
Iteration 4116 out of 5000
Iteration 4117 out of 5000
Iteration 4118 out of 5000
Iteration 4119 out of 5000
Iteration 4120 out of 5000
Iteration 4121 out of 5000
Iteration 4122 out of 5000
Iteration 4123 out of 5000
Iteration 4124 out of 5000
Iteration 4125 out of 5000
Iteration 4126 out of 5000
Iteration 4127 out of 5000
Iteration 4128 out of 5000
Iteration 4129 out of 5000
Iteration 4130 out of 5000
Iteration 4131 out of 5000
Iteration 4132 out of 5000
Iteration 4133 out of 5000
Iteration 4134 out of 5000
Iteration 4135 out of 5000
Iteration 4136 out of 5000
Iteration 4137 out of 5000
Iteration 4138 out of 5000
Iteration 4139 out of 5000
Iteration 4140 out of 5000
Iteration 4141 out of 5000
Iteration 4142 out of 5000
Iteration 4143 out of 5000
I

Iteration 4456 out of 5000
Iteration 4457 out of 5000
Iteration 4458 out of 5000
Iteration 4459 out of 5000
Iteration 4460 out of 5000
Iteration 4461 out of 5000
Iteration 4462 out of 5000
Iteration 4463 out of 5000
Iteration 4464 out of 5000
Iteration 4465 out of 5000
Iteration 4466 out of 5000
Iteration 4467 out of 5000
Iteration 4468 out of 5000
Iteration 4469 out of 5000
Iteration 4470 out of 5000
Iteration 4471 out of 5000
Iteration 4472 out of 5000
Iteration 4473 out of 5000
Iteration 4474 out of 5000
Iteration 4475 out of 5000
Iteration 4476 out of 5000
Iteration 4477 out of 5000
Iteration 4478 out of 5000
Iteration 4479 out of 5000
Iteration 4480 out of 5000
Iteration 4481 out of 5000
Iteration 4482 out of 5000
Iteration 4483 out of 5000
Iteration 4484 out of 5000
Iteration 4485 out of 5000
Iteration 4486 out of 5000
Iteration 4487 out of 5000
Iteration 4488 out of 5000
Iteration 4489 out of 5000
Iteration 4490 out of 5000
Iteration 4491 out of 5000
Iteration 4492 out of 5000
I

Iteration 4800 out of 5000
Iteration 4801 out of 5000
Iteration 4802 out of 5000
Iteration 4803 out of 5000
Iteration 4804 out of 5000
Iteration 4805 out of 5000
Iteration 4806 out of 5000
Iteration 4807 out of 5000
Iteration 4808 out of 5000
Iteration 4809 out of 5000
Iteration 4810 out of 5000
Iteration 4811 out of 5000
Iteration 4812 out of 5000
Iteration 4813 out of 5000
Iteration 4814 out of 5000
Iteration 4815 out of 5000
Iteration 4816 out of 5000
Iteration 4817 out of 5000
Iteration 4818 out of 5000
Iteration 4819 out of 5000
Iteration 4820 out of 5000
Iteration 4821 out of 5000
Iteration 4822 out of 5000
Iteration 4823 out of 5000
Iteration 4824 out of 5000
Iteration 4825 out of 5000
Iteration 4826 out of 5000
Iteration 4827 out of 5000
Iteration 4828 out of 5000
Iteration 4829 out of 5000
Iteration 4830 out of 5000
Iteration 4831 out of 5000
Iteration 4832 out of 5000
Iteration 4833 out of 5000
Iteration 4834 out of 5000
Iteration 4835 out of 5000
Iteration 4836 out of 5000
I